## A/B test

https://developer.amazon.com/public/apis/manage/ab-testing/doc/math-behind-ab-testing

In [58]:
from __future__ import division
import pandas as pd
from numpy import sqrt
import scipy.stats as scipys
import datetime 
import matplotlib.pyplot as plt
%matplotlib inline

In [59]:
def z_test(ctr_old, ctr_new, nobs_old, nobs_new, effect_size=0., two_tailed=True, alpha=.05):
    """Perform z-test to compare two proportions (e.g., Click Through Rates (CTR)).

        Note: if you set two_tailed=False, z_test assumes H_A is that the effect is
        non-negative, so the p-value is computed based on the weight in the upper tail.
        
        Arguments:
            ctr_old (float):    baseline proportion (CTR)
            ctr_new (float):    new proportion
            nobs_old (int):     number of observations in baseline sample
            nobs_new (int):     number of observations in new sample
            effect_size (float):    size of effect
            two_tailed (bool):  True to use two-tailed test; False to use one-sided test
                                where alternative hypothesis if that effect_size is non-negative
            alpha (float):      significance level

        Returns:
            z-score, p-value, and whether to reject the null hypothesis
    """
    # p : conversion rate 
    p = (ctr_old * nobs_old + ctr_new * nobs_new) / (nobs_old + nobs_new)
    
    se = sqrt(p*(1-p)*(1./nobs_old + 1./nobs_new))
    
    z_score = (ctr_new - ctr_old - effect_size) / se
    
    if two_tailed:
        p_val = (1 - scipys.norm.cdf(abs(z_score)))* 2
    else:
        p_val = 1 - scipys.norm.cdf(z_score)
    
    reject_null = p_val < alpha
    print 'p_val:{}, standard_dev:{}, z_score:{}, reject null:{}'.format(p_val, se, z_score, reject_null) 
    
    return p_val, z_score, reject_null
    

In [8]:
df = pd.read_csv('data/experiment.csv')

In [9]:
df[:2]

,user_id,ts,ab,landing_page,converted
0,4040615247,1356998400,treatment,new_page,0
1,4365389205,1356998400,treatment,new_page,0


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 191148 entries, 0 to 191147
Data columns (total 5 columns):
user_id         191148 non-null int64
ts              191148 non-null float64
ab              191148 non-null object
landing_page    191148 non-null object
converted       191148 non-null int64
dtypes: float64(1), int64(2), object(2)
memory usage: 8.8+ MB


In [12]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
user_id,191148,5.007786e+09,2.889032e+09,3416,2.505443e+09,5005865728,7.508326e+09,9999962347
ts,191148,1.357042e+09,2.493174e+04,1356998400,1.357020e+09,1357041629,1.357063e+09,1357084799
converted,191148,9.980225e-02,2.997369e-01,0,0.000000e+00,0,0.000000e+00,1


In [15]:
len(df.user_id.unique())

186388

In [30]:
control_group = df[df['ab'] == 'control'].copy()

In [31]:
control_group[:2]

,user_id,ts,ab,landing_page,converted
3,8122359922,1356998402,control,old_page,0
4,6077269891,1356998402,control,old_page,0


In [32]:
n_control = float(len(control_group.user_id.unique()))

In [33]:
treatment_group = df[df.ab == 'treatment'].copy() 

In [34]:
treatment_group[:2]

,user_id,ts,ab,landing_page,converted
0,4040615247,1356998400,treatment,new_page,0
1,4365389205,1356998400,treatment,new_page,0


In [35]:
n_treatment = float(len(treatment_group.user_id.unique()))

In [36]:
n_treatment

95574.0

In [39]:
conversion_rate_control = control_group.converted.sum() / n_control

In [40]:
conversion_rate_control

0.09964322681524874

In [41]:
conversion_rate_treatment = treatment_group.converted.sum() / n_treatment

In [42]:
conversion_rate_treatment

0.10492393328729571

In [60]:
z_test(conversion_rate_control, conversion_rate_treatment, n_control, n_treatment, two_tailed=False)

p_val:8.51269056524e-05, standard_dev:0.00140463028958, z_score:3.75949921572, reject null:True


(8.512690565243286e-05, 3.7594992157238489, True)

In [63]:
datetime.datetime.fromtimestamp(int(control_group.ts.head(n=1))) #head() is equal than .iloc[0]

datetime.datetime(2012, 12, 31, 16, 0, 2)

In [65]:
control_group['t_stamp'] = control_group.ts.apply(lambda x: datetime.datetime.fromtimestamp(int(x)))

In [67]:
treatment_group['t_stamp'] = treatment_group.ts.apply(lambda x: datetime.datetime.fromtimestamp(int(x)))

In [69]:
a = 4. / sqrt(75)

In [70]:
a

0.46188021535170054

In [73]:
a*1.96

0.90528522208933304

In [74]:
b = 0.05/(1-0.05)

In [75]:
b

0.052631578947368425

In [1]:
#test